Ceci est un exemple de scoring sur une base d'utilisateurs lié à un abonnement téléphonique.

In [1]:
# Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# Load Dataset
data = pd.read_csv("./data/telcoCustomer.csv")

In [3]:
# 
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# data overview
print ("Rows     : " ,data.shape[0])
print ("Columns  : " ,data.shape[1])
print ("\nFeatures : \n" ,data.columns.tolist())
print ("\nUnique values :  \n",data.nunique())
print("\nPourcentage churn : \n", data["Churn"].value_counts(normalize = True))
data.dtypes

Rows     :  7043
Columns  :  21

Features : 
 ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Unique values :  
 customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

Pourcentage churn : 
 No     0.73463
Yes    0.26537
Name: Churn, dtype:

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

**Data Manipulation**

La variable SeniorCitizen est considéré comme une variable continue puisqu'elle contient des 0 et des 1 alors que c'est une variable catégorielle.

In [5]:
data["SeniorCitizen"] = data["SeniorCitizen"].replace({1:"Yes",0:"No"})

**PREPARATION DES DONNEES**

Table des matières :

*I. Missing data*

*II. Valeurs aberrantes*

*III. Valeurs extremes*

*IV. Categorisation*

*V. Discrétisation*

*VI. Croisement*

**I. Valeurs manquantes**

In [6]:
print ("\nMissing values :  ", data.isnull().sum().values.sum())


Missing values :   0


Aucune valeurs manquantes à constater ici, pour autant il existe plusieurs stratégies :

- Enlever la ligne, simple mais on perd de l'information
- Enlever la variable si elle n'est pas jugée vitale
- Remplacer par la moyenne/mediane de la feature. Potentiellement dangereux car les individus ayant des valeurs manquantes sont le plus souvent des individus atypiques (présentant par exemple un risque sup à la moy et dissimulant des informations)
- Régressions, arbre de décision
- Kmeans et on impute la moyenne du groupe



**II. Valeurs aberrantes**

Une valeur aberrante est une valeur erronée correspondant à une mauvaise mesure, une erreur de calcul/saisie ou une fausse déclaration.

Type de valeurs aberrantes :

- date incohérentes 
- numéros de téléphonnes
- sexe prenant plus de deux valeurs différentes
- ...

Pour les traiter, on peut se référer au chapitre sur les valeurs manquantes et appliquer les mêmes idées.

Il faut faire attention ici car "TotalCharges" n'est pas considéré comme une variable continue. Il y a en effet des valeurs manquantes qui ont été transformés en " ".

Appliquons la méthode des moyenne, nous reviendrons dessus plus tard avec une méthode plus robuste.

In [7]:
from sklearn.impute import SimpleImputer
data['TotalCharges'] = data["TotalCharges"].replace(" ",np.nan)

imp_mean = SimpleImputer(missing_values = np.nan, strategy = "mean")
totalCharges = data.loc[:,"TotalCharges"].values
totalCharges = totalCharges.reshape(-1,1)
data.loc[:,"TotalCharges"] = imp_mean.fit_transform(totalCharges).reshape(-1)

**III. Valeurs extrêmes**

Attention ici car une valeur extrême n'est pas forcemment une valeur aberrante. Une valeur extrême peut correspondre à un profil rare, intéressant à détecter, dont la suppression de la population étudiée apauvrirait l'échantillon d'apprentissage et le modèle obtenu.

En dehors de ces cas de recherches de patterns atypiques, il faut traiter ces valeurs car elles affectent certaines méthodes.

On peut :

- écarter ces outliers si cela ne représente pas plus de 1-2%
- discrétiser
- winsoriser : remplacer les valeur de la variable au delà du 99eme centile par ce dernier. Et pareil pour le 1er centile.

In [8]:
data.describe()

,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,2283.300441
std,24.559481,30.090047,2265.000258
min,0.000000,18.250000,18.800000
25%,9.000000,35.500000,402.225000
50%,29.000000,70.350000,1400.550000
75%,55.000000,89.850000,3786.600000
max,72.000000,118.750000,8684.800000


Au vu de nos données, il peut être intéressant de garder nos max car ils ne sont pas excéssifs.

On peut winsoriser "TotalCharges" car le max s'écarte un peu trop de la std mais on pourrait tout autant ne rien toucher.

In [9]:
totalCharges = data["TotalCharges"]
c99 = totalCharges.quantile(0.99)
c1 = totalCharges.quantile(0.01)
totalChargesWin = totalCharges.apply(lambda x: c99 if x > c99 else x)
totalChargesWin = totalCharges.apply(lambda x: c1 if x < c1 else x)
data["TotalCharges"] = totalChargesWin
data.describe()

,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,2283.304665
std,24.559481,30.090047,2264.996036
min,0.000000,18.250000,19.900000
25%,9.000000,35.500000,402.225000
50%,29.000000,70.350000,1400.550000
75%,55.000000,89.850000,3786.600000
max,72.000000,118.750000,8684.800000


**IV. Catégorisation**

On transforme la variable cible 

In [10]:
#customer id col
Id_col = ['customerID']
#Target columns
y = ["Churn"]
#categorical columns
cat_cols = data.nunique()[data.nunique() < 6].keys().tolist()
cat_cols = [x for x in cat_cols if x not in y]
#numerical columns
num_cols   = [x for x in data.columns if x not in cat_cols + y + Id_col]
#Binary columns with 2 values
bin_cols   = data.nunique()[data.nunique() == 2].keys().tolist()
#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

In [43]:
#Label encoding Binary columns
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for i in bin_cols :
    data[i] = le.fit_transform(data[i])

In [ ]:
# Duplicating columns for multi value columns
data = pd.get_dummies(data = data, columns = multi_cols, drop_first = True)
# drop_first Whether to get k-1 dummies out of k categorical levels by removing the first level

In [15]:
#Scaling Numerical columns
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
scaled = std.fit_transform(data[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

/anaconda3/envs/mlenv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by StandardScaler.

/anaconda3/envs/mlenv/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by StandardScaler.



**V. Discrétisation**

**VI. Croisements**

**VII. Echantillonnage**

L'échantillonnage est nécessaire dans l'élaboration d'un modèle prédictif, ne serait-ce que pour la consitution des échantillons d'apprentissages et de test qui permettrons d'optimiser la sélection des variables explicatives.

Cela fait, il faudra revenir à la population entière pour recalculer les paramètres du modèle, non pas sur le seul échantillon d'apprentissage, mais l'ensemble des individus, propre à assurer la meilleure estimation des paramètres du modèle.

On peut distinguer plusieurs méthodes d'échantillonnages aléatoires :

- simple avec proba uniforme
- systématique
- stratifié
- par grappes

In [16]:
from sklearn import model_selection

train, test = model_selection.train_test_split(data,train_size=0.7)

cols    = [i for i in data.columns if i not in Id_col + y]
train_X = train[cols]
train_Y = train[y]
test_X  = test[cols]
test_Y  = test[y]

print(train_Y["Churn"].value_counts(normalize = True))
print(test_Y["Churn"].value_counts(normalize = True))

0    0.738742
1    0.261258
Name: Churn, dtype: float64
0    0.725035
1    0.274965
Name: Churn, dtype: float64


/anaconda3/envs/mlenv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



On s'assure que l'échantillonnage est bien stratifié avec la même proportion de churn dans les échantillons d'apprentissages et de test.

**MODELISATION**

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver = "liblinear")

In [ ]:
from supervised_prediction import supervised_prediction

supervised_prediction(lr, train_X, test_X, train_Y.values.ravel(), test_Y, cols, "coefficients", threshold_plot = True)

Précision = parmi les identifiés positifs, combien étaient réellement positifs ?
Rappel = quelle proportion de résultats positifs réels a été identifiée correctement ?

Le choix d'un indicateur plutot que d'un autre se fera en fonction de l'objectif que l'on veut atteindre. 
Un détecteur de spam voudra être précis alors qu'un détecteur de fraude ou de maladie s'orientera plutot vers le rappel.


In [26]:
from sklearn.feature_selection import RFE

lr = LogisticRegression(solver = "liblinear")
rfe = RFE(lr)
sol = rfe.fit(train_X,train_Y.values.ravel())
print(sol.n_features_) 
print(sol.support_) 

20
[False False False  True False  True  True False False False False False
  True False  True False  True  True  True  True False False  True False
 False  True  True False  True False  True  True False  True  True  True
  True  True False False]


In [29]:
train_X_rfe = train_X.iloc[:,sol.support_]
test_X_rfe = test_X.iloc[:,sol.support_]

In [ ]:
supervised_prediction(lr, train_X_rfe, test_X_rfe, train_Y.values.ravel(), test_Y, cols, "coefficients", threshold_plot = True)

In [45]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,...,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,0,0,1,0,1,0,1,29.85,29.85,...,1,0,0,1,0,0,0,0,1,0
1,5575-GNVDE,1,0,0,0,34,1,0,56.95,1889.50,...,1,0,0,0,1,0,0,0,0,1
2,3668-QPYBK,1,0,0,0,2,1,1,53.85,108.15,...,1,0,0,1,0,0,0,0,0,1
3,7795-CFOCW,1,0,0,0,45,0,0,42.30,1840.75,...,1,0,0,0,1,0,1,0,0,0
4,9237-HQITU,0,0,0,0,2,1,1,70.70,151.65,...,1,0,0,1,0,0,0,0,1,0
